In [2]:
!python -V

Python 3.9.12


In [6]:
import pandas as pd

In [4]:
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error


In [5]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748969976755, experiment_id='1', last_update_time=1748969976755, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
print(len(df))  #How many records did we load?

3403766


In [7]:
def read_dataframe():
    df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return len((df))

In [8]:
print(read_dataframe()) #question-4

3316216


In [ ]:

categorical = ['PULocationID', 'DOLocationID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)



In [11]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration'],
      dtype='object')

In [12]:
target = 'duration'
y_train = df['duration'].values


In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)

In [ ]:
print("Intercept:", lr.intercept_)

In [16]:
from pathlib import Path

In [17]:
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [18]:

import mlflow
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from pathlib import Path

models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

with mlflow.start_run():
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    
    # Log parameters (example: just default params here)
    mlflow.log_param("fit_intercept", lr.fit_intercept)
    
    # Log metrics
    mlflow.log_metric("mse", mse)
    
    # Save the model locally and log it as artifact
    model_path = models_folder / "linear_regression_model.pkl"
    with open(model_path, "wb") as f_out:
        pickle.dump(lr, f_out)
    mlflow.log_artifact(str(model_path))



🏃 View run rumbling-tern-962 at: http://localhost:5000/#/experiments/1/runs/24ea153676c040d2bb6521244eca26d3
🧪 View experiment at: http://localhost:5000/#/experiments/1
